In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

In [2]:
import torch
import torch.nn as nn
import numpy as np
import time
import datetime
import wandb
import gc
import pandas as pd
from train_helper import train_fn, valid_fn
from config import TrainConfig
from utils import get_logger, get_scheduler, load_pkl, weight_init, save_pkl
from dltime.data.ts_datasets import Soil_Dataset
from dltime.models.inception_atten import TSInceptionSelfAttnEncoderClassifier
from dltime.models.conv_atten import TSTransformerEncoderConvClassifier
from transformers import AdamW
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils import shuffle
from data_process import handle_dataset_3dims

/home/shizhaoshu/.conda/envs/torch111/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CFG = TrainConfig()

In [4]:
# data process
data_for_train = ['zwy', 'zwy2', 'zwy3', 'zwy4']
train_data = []
for data_name in data_for_train:
    train_data.extend(load_pkl(f'./pickle_data/{data_name}_train.pkl'))

train_x, train_label = handle_dataset_3dims(train_data, mode="all")
train_x = np.swapaxes(train_x, 2, 1)
print('Train data size:', train_x.shape, train_label.shape)

train_dataset = Soil_Dataset(train_x, train_label, normalize=None, channel_first=True)
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=False)

Train data size: (4672, 48, 5) (4672,)


In [5]:
model = TSInceptionSelfAttnEncoderClassifier(
        feat_dim=5, 
        max_len=64, 
        d_model=512, 
        num_heads=4,
        num_layers=4,
        dim_feedforward=None,
        num_classes=3).to(CFG.device)

model.load_state_dict(torch.load('./outputs/zwy_2022-05-24 16.38.pth'))

<All keys matched successfully>

In [6]:
model.eval()
preds = []
start = end = time.time()
for step, item in enumerate(train_loader):
    for k, v in item.items():
        item[k] = v.to(CFG.device)

    labels = item['label']
    batch_size = labels.size(0)
    with torch.no_grad():
        y_preds = model(item['input'])

    preds.append(y_preds.cpu().numpy())

predictions = np.concatenate(preds)

In [7]:
save_pkl(predictions, './soil_outputs/zwy_prebs.pkl')

In [8]:
prebs = load_pkl('./soil_outputs/zwy_prebs.pkl')

In [9]:
train_label

array([2, 2, 2, ..., 0, 0, 0])